# Animal Movement Analysis Notebook Documentation

## Overview
This notebook analyzes animal movement data in relation to protected areas, focusing on boundary crossings, spatial patterns, and temporal trends. The analysis uses Python libraries such as pandas, geopandas, and folium for data manipulation, geospatial analysis, and visualization.

## Data Sources
1. **Animal Events Data** (`animal_events.csv`): Contains timestamped location data for various animals.
2. **Animals Data** (`animals.csv`): Provides information about each animal, including species and conservation status.
3. **Protected Areas Data** (`protected_areas.json`): Geospatial data defining the boundaries of protected areas.
4. **Satellite Data** (`satellites.json`): Information about satellite coverage, including bounding boxes and temporal ranges.

## Key Libraries Used
- `pandas`: Data manipulation and analysis
- `geopandas`: Geospatial data handling
- `folium`: Interactive map creation
- `matplotlib`: Static visualizations
- `sklearn`: For clustering analysis (DBSCAN)

## Main Analyses

### 1. Data Preparation and Exploration
- Loading and initial exploration of animal events, animal information, and protected areas data.
- Merging datasets to create a comprehensive view of animal movements.

### 2. Geospatial Analysis
- Creating GeoDataFrames for animal locations and protected areas.
- Identifying whether animal locations are within protected areas.

### 3. Visualization
- Interactive maps showing:
  - Protected area boundaries
  - Animal locations and movements
  - Clusters of animal activity
- Heatmaps or density plots of animal presence

### 4. Boundary Crossing Analysis
- Identifying and quantifying boundary crossings into and out of protected areas.
- Analyzing crossings by time of day and day of week.
- Calculating total entries and exits from protected areas.

### 5. Temporal Patterns
- Analyzing the percentage of time animals spend in protected areas.
- Investigating if multiple animals tend to cross boundaries together.

### 6. Spatial Patterns
- Using DBSCAN clustering to identify areas of high animal density.
- Analyzing if certain protected areas have more crossings than others.

### 7. Individual Animal Analysis
- Identifying animals with frequent boundary crossings.
- Calculating average time between crossings for each animal.

### 8. Predictive Modeling
- Simple predictive model using last known location as a prediction for the next location.
- Calculating prediction errors to assess model performance.

## Key Findings
- Total number of boundary crossings and their distribution across time.
- Percentage of time animals spend in protected areas.
- Identification of potential risk zones (areas of high animal density).
- Patterns in boundary crossings (e.g., time of day, day of week, multiple animals crossing together).
- Animals with the most frequent boundary crossings and their average time between crossings.

## Visualizations
- Interactive maps showing protected areas, animal movements, and clusters.
- Plots of boundary crossings by hour and day of week.
- Heatmaps or density plots of animal presence.

## Future Work
- Incorporate more sophisticated predictive models for animal movement.
- Analyze seasonal patterns in animal movements and boundary crossings.
- Investigate correlations between animal movements and environmental factors.
- Expand the analysis to include more species or larger geographical areas.


In [90]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
import geodatasets

In [3]:
animal_events: pd.DataFrame = pd.read_csv(
    "../../data/animal_events.csv",
    parse_dates=["timestamp"],
    dtype={"animal_id": str, "latitude": float, "longitude": float},
)
animal_events.head(5)

,animal_id,timestamp,latitude,longitude
0,A001,2024-09-01 12:00:00,45.2284,-110.7622
1,A002,2024-09-01 12:00:00,44.5760,-110.6763
2,A003,2024-09-01 12:00:00,44.4232,-111.1061
3,A004,2024-09-01 12:00:00,37.9058,-119.7857
4,A005,2024-09-01 12:00:00,37.7896,-119.6426


In [12]:
pd.DataFrame(animal_events.isna().mean())

,0
animal_id,0.0
timestamp,0.0
latitude,0.0
longitude,0.0


In [15]:
animals: pd.DataFrame = pd.read_csv("../../data/animals.csv")
animals = animals[["animal_id", "common_name", "redlist_cat", "megafauna"]]
animals.head()

,animal_id,common_name,redlist_cat,megafauna
0,A001,Wolf,Least Concern,no
1,A002,Bison,Vulnerable,yes
2,A003,Elk,Least Concern,yes
3,A004,Sierra Nevada bighorn sheep,Endangered,no
4,A005,Sierra Nevada red fox,Critically Endangered,no


In [67]:
animals_geoloc_events = pd.merge(animal_events, animals, on="animal_id")
animals_geoloc_events.head()

,animal_id,timestamp,latitude,longitude,common_name,redlist_cat,megafauna
0,A001,2024-09-01 12:00:00,45.2284,-110.7622,Wolf,Least Concern,no
1,A002,2024-09-01 12:00:00,44.5760,-110.6763,Bison,Vulnerable,yes
2,A003,2024-09-01 12:00:00,44.4232,-111.1061,Elk,Least Concern,yes
3,A004,2024-09-01 12:00:00,37.9058,-119.7857,Sierra Nevada bighorn sheep,Endangered,no
4,A005,2024-09-01 12:00:00,37.7896,-119.6426,Sierra Nevada red fox,Critically Endangered,no


In [60]:
satelite_data = pd.read_json("../../data/satellites.json")
satelite_data["bbox_geometry"] = satelite_data["bounding_box"].apply(
    lambda x: box(x["xmin"], x["ymin"], x["xmax"], x["ymax"])
)
satelite_data["center_geometry"] = satelite_data["bbox_geometry"].apply(
    lambda x: x.centroid
)
satelite_data = gpd.GeoDataFrame(
    satelite_data, geometry="bbox_geometry", crs="EPSG:4326"
)
satelite_data

,satellite_id,start_time,last_time,frequency,bounding_box,cloud_cover_percentage,resolution,bbox_geometry,center_geometry
0,SAT001,2018-09-01 12:00:00+00:00,2024-09-10 12:00:00+00:00,daily,"{'xmin': -112.939131, 'ymin': 42.596356, 'xmax...",12.5,10m,"POLYGON ((-107.04873 42.59636, -107.04873 46.1...",POINT (-109.99392850000001 44.369389999999996)
1,SAT002,2004-09-01 12:00:00+00:00,2024-09-06 12:00:00+00:00,bi-weekly,"{'xmin': -180, 'ymin': -90, 'xmax': 180, 'ymax...",10.0,100m,"POLYGON ((180 -90, 180 90, -180 90, -180 -90, ...",POINT (0 0)
2,SAT003,2022-09-01 12:00:00+00:00,2024-09-10 12:00:00+00:00,hourly,"{'xmin': -124.178099, 'ymin': 30.738207, 'xmax...",10.0,20m,"POLYGON ((-95.94283 30.73821, -95.94283 51.538...",POINT (-110.06046500000001 41.13856800000001)


In [96]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import contextily as cx
from sklearn.preprocessing import StandardScaler
import folium
from folium import FeatureGroup, LayerControl
from branca.colormap import LinearColormap

In [84]:
# Load the protected areas data
with open("../../data/protected_areas.json") as f:
    protected_areas_data = json.load(f)

# Create a GeoDataFrame for protected areas
protected_areas = gpd.GeoDataFrame.from_features(protected_areas_data["features"])
# protected_areas['geometry'] = protected_areas['geometry'].apply(lambda x: Polygon(x['coordinates'][0]))
# Create a GeoDataFrame for animal events
geometry = [
    Point(xy) for xy in zip(animal_events["longitude"], animal_events["latitude"])
]
animal_events_gdf = gpd.GeoDataFrame(
    animals_geoloc_events, geometry=geometry, crs="EPSG:4326"
)

# Exploratory Data Analysis
print("Exploratory Data Analysis:")
pd.DataFrame(animal_events.describe().T)

Exploratory Data Analysis:


,count,mean,min,25%,50%,75%,max,std
timestamp,27,2024-09-01 13:00:00,2024-09-01 12:00:00,2024-09-01 12:00:00,2024-09-01 13:00:00,2024-09-01 14:00:00,2024-09-01 14:00:00,NaN
latitude,27.0,39.622411,36.2261,36.62375,37.8626,44.4089,45.2284,3.594347
longitude,27.0,-114.293944,-119.7857,-119.62675,-112.3388,-110.9015,-110.1118,3.947671


In [85]:
# print("\nUnique animals:", animal_events['animal_id'].nunique())
# print("Time range:", animal_events['timestamp'].min(), "to", animal_events['timestamp'].max())

In [119]:
import folium.plugins

# Create an interactive map
m = folium.Map(
    location=[animal_events_gdf.geometry.y.mean(), animal_events_gdf.geometry.x.mean()],
    zoom_start=6,
)

# Add zoom control
folium.plugins.Fullscreen().add_to(m)
folium.plugins.MousePosition().add_to(m)

# Add protected areas to the map
protected_areas_layer = FeatureGroup(name="Protected Areas")
for idx, row in protected_areas.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x: {
            "fillColor": "green",
            "color": "black",
            "weight": 2,
            "fillOpacity": 0.4,
        },
        tooltip=row["name"],
    ).add_to(protected_areas_layer)
protected_areas_layer.add_to(m)

# Add animal locations to the map
animal_locations_layer = FeatureGroup(name="Animal Locations")
for idx, row in animal_events_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        popup=f"Animal ID: {row['animal_id']}, Time: {row['timestamp']}",
        color="red",
        fill=True,
        fillColor="red",
    ).add_to(animal_locations_layer)
animal_locations_layer.add_to(m)

# Add layer control
LayerControl().add_to(m)

# Display the map
display(m)


# Behavioral Analysis
def is_point_in_polygon(point, polygon):
    return polygon.contains(point)


animal_events_gdf["in_protected_area"] = animal_events_gdf.apply(
    lambda row: any(
        is_point_in_polygon(row.geometry, area) for area in protected_areas.geometry
    ),
    axis=1,
)

# Identify crossings
crossings = (
    animal_events_gdf.groupby("animal_id")["in_protected_area"].diff().abs().sum()
)
print("\nBehavioral Analysis:")
print(f"Total number of boundary crossings: {crossings}")

# Advanced Insights
# Clustering analysis
coords = animal_events_gdf[["geometry"]]
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(
    coords.geometry.apply(lambda p: [p.x, p.y]).tolist()
)

dbscan = DBSCAN(eps=0.1, min_samples=2)
animal_events_gdf["cluster"] = dbscan.fit_predict(coords_scaled)

print("\nAdvanced Insights:")
print(f"Number of clusters identified: {animal_events_gdf['cluster'].max() + 1}")

# Create a map for clusters
m_clusters = folium.Map(
    location=[animal_events_gdf.geometry.y.mean(), animal_events_gdf.geometry.x.mean()],
    zoom_start=6,
)

# Add zoom control to cluster map
folium.plugins.Fullscreen().add_to(m_clusters)
folium.plugins.MousePosition().add_to(m_clusters)

# Add protected areas to the cluster map
protected_areas_layer.add_to(m_clusters)

# Create a color map for clusters
color_map = LinearColormap(
    colors=["red", "blue", "green", "purple", "orange"],
    vmin=animal_events_gdf["cluster"].min(),
    vmax=animal_events_gdf["cluster"].max(),
)

# Add clustered animal locations to the map
for idx, row in animal_events_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        popup=f"Animal ID: {row['animal_id']}, Cluster: {row['cluster']}",
        color=color_map(row["cluster"]),
        fill=True,
        fillColor=color_map(row["cluster"]),
    ).add_to(m_clusters)

# Add color map to the map
color_map.add_to(m_clusters)

# Display the cluster map
display(m_clusters)

# Simple predictive model (using last known location as prediction)
animal_events_gdf["next_longitude"] = animal_events_gdf.groupby("animal_id")[
    "longitude"
].shift(-1)
animal_events_gdf["next_latitude"] = animal_events_gdf.groupby("animal_id")[
    "latitude"
].shift(-1)

# Calculate prediction error
animal_events_gdf["prediction_error"] = (
    (animal_events_gdf["longitude"] - animal_events_gdf["next_longitude"]) ** 2
    + (animal_events_gdf["latitude"] - animal_events_gdf["next_latitude"]) ** 2
) ** 0.5

print(
    f"Average prediction error: {animal_events_gdf['prediction_error'].mean():.4f} degrees"
)

# Identify potential risk zones (areas with high animal density)
risk_zones = (
    animal_events_gdf[animal_events_gdf["cluster"] != -1]
    .groupby("cluster")
    .agg({"longitude": "mean", "latitude": "mean", "animal_id": "count"})
    .rename(columns={"animal_id": "animal_count"})
    .sort_values("animal_count", ascending=False)
)

print("\nPotential risk zones (high animal density areas):")
print(risk_zones)

# Create a map for risk zones
m_risk = folium.Map(
    location=[animal_events_gdf.geometry.y.mean(), animal_events_gdf.geometry.x.mean()],
    zoom_start=6,
)

# Add zoom control to risk zone map
folium.plugins.Fullscreen().add_to(m_risk)
folium.plugins.MousePosition().add_to(m_risk)

# Add protected areas to the risk map
protected_areas_layer.add_to(m_risk)

# Add risk zones to the map
for idx, row in risk_zones.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=row["animal_count"],
        popup=f"Cluster: {idx}, Animal Count: {row['animal_count']}",
        color="red",
        fill=True,
        fillColor="red",
        fillOpacity=0.7,
    ).add_to(m_risk)

# Display the risk zone map
display(m_risk)


Behavioral Analysis:
Total number of boundary crossings: 3

Advanced Insights:
Number of clusters identified: 3


Average prediction error: 0.3242 degrees

Potential risk zones (high animal density areas):
          longitude   latitude  animal_count
cluster                                     
1       -119.672644  37.824089             9
2       -112.368162  36.514025             8
0       -110.842957  44.464443             7


In [116]:
# Create a map for migration paths
m_migration = folium.Map(
    location=[animal_events_gdf.geometry.y.mean(), animal_events_gdf.geometry.x.mean()],
    zoom_start=6,
)

# Add zoom control to migration map
folium.plugins.Fullscreen().add_to(m_migration)
folium.plugins.MousePosition().add_to(m_migration)

# Add protected areas to the migration map
protected_areas_layer.add_to(m_migration)

# Create a color map for different animals
unique_animals = animal_events_gdf["animal_id"].unique()

# Add migration paths to the map
for i, animal_id in enumerate(unique_animals):
    animal_data = animal_events_gdf[
        animal_events_gdf["animal_id"] == animal_id
    ].sort_values("timestamp")

    # Create a feature group for each animal
    animal_layer = FeatureGroup(name=f"Animal {animal_id}")

    # Add path
    locations = animal_data[["latitude", "longitude"]].values.tolist()
    folium.PolyLine(
        locations=locations,
        color=color_map(i),
        weight=2,
        opacity=0.8,
        popup=f"Animal {animal_id}",
    ).add_to(animal_layer)

    # Add markers for start and end points
    folium.CircleMarker(
        location=locations[0],
        radius=5,
        popup=f"Animal {animal_id} Start",
        color=color_map(i),
        fill=True,
        fillColor=color_map(i),
    ).add_to(animal_layer)

    folium.CircleMarker(
        location=locations[-1],
        radius=5,
        popup=f"Animal {animal_id} End",
        color=color_map(i),
        fill=True,
        fillColor=color_map(i),
    ).add_to(animal_layer)

    animal_layer.add_to(m_migration)

# Add layer control to migration map
LayerControl().add_to(m_migration)

# Add color map legend to the map
color_map.add_to(m_migration)
color_map.caption = "Animal IDs"

# Display the migration path map
display(m_migration)

In [121]:
# Identify boundary crossings
animal_events_gdf["boundary_crossing"] = (
    animal_events_gdf.groupby("animal_id")["in_protected_area"].diff() != 0
)

# Calculate total number of crossings
total_crossings = animal_events_gdf["boundary_crossing"].sum()
print(f"Total number of boundary crossings: {total_crossings}")

# Analyze crossings by time of day
animal_events_gdf["hour"] = animal_events_gdf["timestamp"].dt.hour
crossings_by_hour = (
    animal_events_gdf[animal_events_gdf["boundary_crossing"]].groupby("hour").size()
)
print("\nCrossings by hour of day:")
print(crossings_by_hour)

# Analyze crossings by day of week
animal_events_gdf["day_of_week"] = animal_events_gdf["timestamp"].dt.day_name()
crossings_by_day = (
    animal_events_gdf[animal_events_gdf["boundary_crossing"]]
    .groupby("day_of_week")
    .size()
)
print("\nCrossings by day of week:")
print(crossings_by_day)

# Calculate entries and exits
entries = animal_events_gdf[
    animal_events_gdf["boundary_crossing"] & animal_events_gdf["in_protected_area"]
].shape[0]
exits = animal_events_gdf[
    animal_events_gdf["boundary_crossing"] & ~animal_events_gdf["in_protected_area"]
].shape[0]
print(f"\nTotal entries into protected areas: {entries}")
print(f"Total exits from protected areas: {exits}")

# Analyze time spent in protected areas
animal_events_gdf["time_diff"] = animal_events_gdf.groupby("animal_id")[
    "timestamp"
].diff()
time_in_protected = animal_events_gdf[animal_events_gdf["in_protected_area"]][
    "time_diff"
].sum()
total_time = animal_events_gdf["time_diff"].sum()
percentage_time_in_protected = (time_in_protected / total_time) * 100

print(
    f"\nPercentage of time spent in protected areas: {percentage_time_in_protected:.2f}%"
)

# Analyze if animals tend to cross boundaries together
animal_events_gdf["crossing_window"] = animal_events_gdf["timestamp"].dt.floor("H")
crossings_per_window = (
    animal_events_gdf[animal_events_gdf["boundary_crossing"]]
    .groupby("crossing_window")
    .size()
)
multiple_crossings = (crossings_per_window > 1).sum()
percentage_multiple_crossings = (
    multiple_crossings / crossings_per_window.shape[0]
) * 100

print(
    f"\nPercentage of crossing events with multiple animals: {percentage_multiple_crossings:.2f}%"
)

# Analyze if certain protected areas have more crossings
if "name" in protected_areas.columns:
    animal_events_gdf["protected_area_name"] = animal_events_gdf.apply(
        lambda row: (
            protected_areas[protected_areas.contains(row.geometry)]["name"].values[0]
            if row["in_protected_area"]
            else "Outside"
        ),
        axis=1,
    )
    crossings_by_area = (
        animal_events_gdf[animal_events_gdf["boundary_crossing"]]
        .groupby("protected_area_name")
        .size()
    )
    print("\nCrossings by protected area:")
    print(crossings_by_area)

# Identify animals with frequent boundary crossings
frequent_crossers = (
    animal_events_gdf[animal_events_gdf["boundary_crossing"]]
    .groupby("animal_id")
    .size()
    .sort_values(ascending=False)
)
print("\nAnimals with most frequent boundary crossings:")
print(frequent_crossers.head())

# Calculate the average time between crossings for each animal
time_between_crossings = (
    animal_events_gdf[animal_events_gdf["boundary_crossing"]]
    .groupby("animal_id")["time_diff"]
    .mean()
)
print("\nAverage time between crossings for each animal (in hours):")
print(time_between_crossings.dt.total_seconds() / 3600)

Total number of boundary crossings: 12

Crossings by hour of day:
hour
12    9
14    3
dtype: int64

Crossings by day of week:
day_of_week
Sunday    12
dtype: int64

Total entries into protected areas: 3
Total exits from protected areas: 9

Percentage of time spent in protected areas: 16.67%

Percentage of crossing events with multiple animals: 100.00%

Crossings by protected area:
protected_area_name
Grand Canyon National Park    1
Outside                       9
Yellowstone National Park     1
Yosemite National Park        1
dtype: int64

Animals with most frequent boundary crossings:
animal_id
A002    2
A007    2
A004    2
A001    1
A003    1
dtype: int64

Average time between crossings for each animal (in hours):
animal_id
A001    NaN
A002    1.0
A003    NaN
A004    1.0
A005    NaN
A006    NaN
A007    1.0
A008    NaN
A009    NaN
Name: time_diff, dtype: float64


/tmp/ipykernel_85530/2804193921.py:35: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  animal_events_gdf['crossing_window'] = animal_events_gdf['timestamp'].dt.floor('H')
